# Import Libraries

In [19]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import dalex as dx

from copy import copy

{
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "matplotlib": matplotlib.__version__,
    "seaborn": sns.__version__,
    "sklearn": sklearn.__version__,
    "dalex": dx.__version__,
}

{'numpy': '2.3.3',
 'pandas': '2.3.2',
 'matplotlib': '3.10.6',
 'seaborn': '0.13.2',
 'sklearn': '1.7.2',
 'dalex': '1.7.2'}

In [21]:
df = pd.read_csv("./stackoverflow_full.csv", index_col=0)
target = "Employed"

## ML prerequisites

In [22]:
#split your data set in 2 parts : training and testing

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.3,
    random_state=42
)

In [23]:
# Protected attribute is 0 if a man or non binary and 0 if a woman plus the age

protected = (pd.Series(np.where(X_test["Gender"] == "Woman", '1', '0'), index=X_test.index) 
             + '_' 
             + X_test.Age)
protected_train = (pd.Series(np.where(X_train["Gender"] == "Woman", '1', '0').astype(str), index=X_train.index) 
                   + '_' 
                   + X_train.Age)

# Privileged population is men under 35 years old
privileged = '0_<35'

In [24]:
preprocessor = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

#You can change the Decision tree hyperparameters or the classifier below

clf_decisiontree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, random_state=123))
])

In [25]:
# clf_decisiontree.fit(df.drop(columns=[target]), df[target])
clf_decisiontree.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('passthrough', ...), ('onehotencoder', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [26]:
# exp_decisiontree = dx.Explainer(clf_decisiontree, df.drop(columns=[target]), df[target], verbose=False)
exp_decisiontree = dx.Explainer(clf_decisiontree, X_test, y_test, verbose=True)

Preparation of a new explainer is initiated

  -> data              : 22039 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 22039 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x00000216090F8D60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.536, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = -0.000922, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!


In [27]:
exp_decisiontree.model_performance().result

,recall,precision,f1,accuracy,auc
DecisionTreeClassifier,0.804868,0.788158,0.796425,0.779845,0.860599


In [28]:
fairness_decisiontree = exp_decisiontree.model_fairness(protected=protected, privileged=privileged)

In [29]:
fairness_decisiontree.fairness_check(epsilon = 0.8) # default epsilon

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0_<35'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
0_>35  0.969325  0.989770  0.982346  0.942085  0.937943
1_<35  0.947239  1.006394  0.979823  0.768340  0.833333
1_>35  0.900613  1.046036  1.034048  0.459459  0.673759


In [30]:
fairness_decisiontree.plot(verbose=False)

c:\Users\chahb\Desktop\rentré\Hackathon\Notebooks (pre and post processing)\.venv\Lib\site-packages\dalex\fairness\_group_fairness\utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Strategy 1.2 Pre-processing: Remove sensitive attribute

The first thing that can come to mind is to remove sensitive variables. However, this option is considered as really naive since it can have no effect. 

**When may this method works ?**

➤ If the sensitive variable conveys the bias (mostly) on its own.

    ➤ This means, the sensitive variable is correlated with the target variable
    ➤ This also means, the sensitive variable is NOT correlated at all with other explanatory variables and any combination of other explanatory variables cannot be used as a proxi for the sensitive variable. 

Most of the time, the bias is shared accross explanatory variables. For example, a bias on gender may be present in many ones (salary, education, socio-professional category, etc.)

This transformation is not possible in the dalex module (since it not a recommended option to deal with bias). To implement it, a new model has to be trained and explained. This time the sensitive variable

#### Training

In [35]:
# Retrain a model without sensitive variables "age" and "sex"
X_train_restricted = X_train.drop(columns=["Gender", "Age"], errors="ignore")


#we also make the ComputerSkills column categorical
# bins = [0, 2, 10, 25, float('inf')]
# labels = ['null', 'faible', 'moyen', 'élevé']
# X_train_restricted['ComputerSkills_level'] = pd.cut(X_train_restricted['ComputerSkills'], bins=bins, labels=labels, right=False)
# X_test['ComputerSkills_level'] = pd.cut(X_test['ComputerSkills'], bins=bins, labels=labels, right=False)

# X_train_restricted = X_train_restricted.drop(columns=["ComputerSkills"], errors="ignore")
# X_test = X_test.drop(columns=["ComputerSkills"], errors="ignore")



preprocessor_restr = make_column_transformer(
      ("passthrough", make_column_selector(dtype_include=np.number)),
      (OneHotEncoder(handle_unknown="ignore"), make_column_selector(dtype_include=object))
)

clf_decisiontree_restr = Pipeline(steps=[
    ('preprocessor', preprocessor_restr),
    ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
])

clf_decisiontree_restr.fit(X_train_restricted, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('passthrough', ...), ('onehotencoder', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


#### Algorithmic performance

In [36]:
# Create a new dalex explainer for the model without sensitive variables
exp_decisiontree_restr = dx.Explainer(clf_decisiontree_restr, X_test, y_test, verbose=True)

exp_decisiontree_restr.model_performance().result

Preparation of a new explainer is initiated

  -> data              : 22039 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 22039 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x00000216090F8D60> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.536, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = -0.000585, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!


,recall,precision,f1,accuracy,auc
DecisionTreeClassifier,0.816825,0.783727,0.799934,0.781388,0.867263



When comparing these performance metrics with those of the model with all variables, what can you observe?

In our case, can we use this metric to compare models?

#### Fairness performance
    

In [37]:
# Let's see effect of removing sensitive variables on bias metrics
fairness_decisiontree_restr = exp_decisiontree_restr.model_fairness(protected=protected, privileged=privileged, 
                                                                    label='DecisionTreeClassifier_no_sensitive')

fairness_decisiontree_restr.fairness_check(epsilon = 0.8) # default epsilon

fairness_decisiontree_restr.plot()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0_<35'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
0_>35  0.957882  0.993614  0.994911  0.880000  0.915517
1_<35  0.939832  1.003831  0.979644  0.760000  0.827586
1_>35  0.864019  1.001277  0.968193  0.607273  0.691379


c:\Users\chahb\Desktop\rentré\Hackathon\Notebooks (pre and post processing)\.venv\Lib\site-packages\dalex\fairness\_group_fairness\utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



What impact does the pre-processing startegy had on the bias?

In [38]:
# Compare 2 (or more) fairness objects in dalex (add them as list in parameters). 
# It's also possible to choose the plot type ! 
fairness_decisiontree_restr.plot([fairness_decisiontree], type='radar')